In [4]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Carregar o dataset
df1 = pd.read_csv('Subsampling.csv')

# Convertendo a coluna 'Data' para datetime
df1['Data'] = pd.to_datetime(df1['Data'], format='%Y-%m-%d')

# Criando uma coluna datetime completa combinando data e hora
df1['datetime'] = pd.to_datetime(df1['Data'].astype(str) + ' ' + df1['Hora (UTC)'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

# Transformar datetime em uma coluna numérica (timestamp float)
df1['timestamp'] = df1['datetime'].view('int64') / 1e9  # Converte para segundos (float)

# Removendo colunas desnecessárias para o modelo
df1.drop(columns=['Data', 'Hora (UTC)', 'datetime'], inplace=True)

# Certificando que todos os tipos de dados sejam float64 para normalização adequada
df1 = df1.astype(np.float64)

# Separar as features (X) e o alvo (y)
X = df1.drop(columns=['num_ocorrencias'])
y = df1['num_ocorrencias'].values

# Carregar o modelo salvo
modelo_caminho = 'Modelo_A64-S87-AUC64-E40.keras'
modelo = tf.keras.models.load_model(modelo_caminho)

# Normalizar as features
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

# Normalizar o target
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Reformatar X para [amostras, timesteps, características] - ajustando para LSTM
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Fazer previsões no conjunto de teste
y_pred = modelo.predict(X_test)

# Reverter a transformação dos valores para a escala original
try:
    y_pred = scaler_y.inverse_transform(y_pred)
    y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
except NameError:
    print("Scalers não definidos; usando previsões e testes sem transformação inversa.")

# Definir um limiar para converter previsões contínuas em classes binárias
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(float)
y_test_binary = (y_test > threshold).astype(float)

# Avaliação do modelo
accuracy = accuracy_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
roc_auc = roc_auc_score(y_test_binary, y_pred_binary)
conf_matrix = confusion_matrix(y_test_binary, y_pred_binary)

# Cálculo da especificidade
tn, fp, fn, tp = conf_matrix.ravel()
specificity = tn / (tn + fp)

# Exibir resultados de avaliação
print(f'Acurácia: {accuracy:.4f}')
print(f'Sensibilidade (Recall): {recall:.4f}')
print(f'AUC: {roc_auc:.4f}')
print(f'Especificidade: {specificity:.4f}')

# Avaliar o modelo com os dados de teste
avaliacao = modelo.evaluate(X_test, y_test)
print(f"Avaliação do Modelo: {avaliacao}")

# Fazer previsões com o modelo
previsoes = modelo.predict(X_test)

# Para classificação binária, comparar com o threshold
if previsoes.shape[1] == 1:
    previsoes_classes = (previsoes > threshold).astype(int)
else:
    previsoes_classes = previsoes.argmax(axis=1)  # Para classificação com mais de duas classes

# Gerar relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test_binary, previsoes_classes))

# Matriz de Confusão
print("Matriz de Confusão:")
print(conf_matrix)


C:\Users\Levi\AppData\Local\Temp\ipykernel_125412\3468391338.py:18: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df1['timestamp'] = df1['datetime'].view('int64') / 1e9  # Converte para segundos (float)


211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Acurácia: 0.6453
Sensibilidade (Recall): 0.8754
AUC: 0.6402
Especificidade: 0.4050
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4956 - loss: 1.9748
Avaliação do Modelo: [2.0147488117218018, 0.48929527401924133]
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.49      1.00      0.66      3291
         1.0       0.00      0.00      0.00      3435

    accuracy                           0.49      6726
   macro avg       0.24      0.50      0.33      6726
weighted avg       0.24      0.49      0.32      6726

Matriz de Confusão:
[[1333 1958]
 [ 428 3007]]


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import tensorflow as tf

# Caminho do modelo salvo
modelo_caminho = 'Modelo_A64-S87-AUC64-E40.keras'

# Carregar o modelo treinado
modelo = tf.keras.models.load_model(modelo_caminho)


PREPARAÇAO

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Carregar os novos dados (substitua pelo caminho correto dos dados)
novos_dados = pd.read_csv('novos_dados.csv')

# Convertendo as colunas de data e hora (ajuste conforme a estrutura dos novos dados)
novos_dados['Data'] = pd.to_datetime(novos_dados['Data'], format='%Y-%m-%d')
novos_dados['datetime'] = pd.to_datetime(novos_dados['Data'].astype(str) + ' ' + novos_dados['Hora (UTC)'].astype(str) + ':00')
novos_dados['timestamp'] = novos_dados['datetime'].astype('int64') / 1e9

# Remover colunas que não são necessárias para a previsão
novos_dados.drop(columns=['Data', 'Hora (UTC)', 'datetime'], inplace=True)

# Garantir que todas as colunas sejam do tipo float64
novos_dados = novos_dados.astype(np.float64)

# Selecionar as features (X) para predição
X_novos = novos_dados.drop(columns=['num_ocorrencias'], errors='ignore')  # Remova 'num_ocorrencias' se presente

# Normalizar os novos dados com o mesmo scaler usado no treinamento
X_novos_scaled = scaler_X.transform(X_novos)  # Certifique-se que 'scaler_X' está carregado do treino

# Ajustar a forma do X para o formato LSTM [samples, timesteps, features]
X_novos_scaled = X_novos_scaled.reshape((X_novos_scaled.shape[0], 1, X_novos_scaled.shape[1]))


PREDIÇAO

In [ ]:
# Fazer previsões com os novos dados processados
previsoes = modelo.predict(X_novos_scaled)

# Desnormalizar as previsões se necessário
previsoes_desnormalizadas = scaler_y.inverse_transform(previsoes)

# Aplicar o limiar para converter previsões contínuas em binárias
threshold = 0.5 
previsoes_binarias = (previsoes_desnormalizadas > threshold).astype(int)

# Exibir resultados
print("Previsões Binárias (0 = Sem Queda, 1 = Com Queda):")
print(previsoes_binarias)


API  com Flask

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Inicializa o Flask
app = Flask(__name__)

# Carrega o modelo e scalers
modelo = tf.keras.models.load_model('Modelo_A64-S87-AUC64-E40.keras')
scaler_X = MinMaxScaler()  # Assumindo que esses scalers foram salvos e carregados corretamente
scaler_y = MinMaxScaler()

@app.route('/prever', methods=['POST'])
def prever():
    dados = request.json
    df = pd.DataFrame(dados)
    # (Pre-processamento como nos passos anteriores)
    # Fazer a predição e retornar o resultado
    previsoes = modelo.predict(df)  # Ajuste conforme necessário
    return jsonify(previsoes.tolist())

# Executa o app
if __name__ == '__main__':
    app.run(debug=True)
